In [ ]:
#!pip install pytesseract 
#!pip install opencv 
#!pip install numpy as np
#!pip install pillow
#!pip install postal

In [151]:
import pytesseract 
from PIL import Image, ImageOps
import numpy as np
# from postal.parser import parse_address

In [152]:

def extract_text(url):
    import pytesseract 
    from PIL import Image
    
    """extracts test out of image
    --inputs= image_url """
    #opening the image
    im=Image.open(url)
    grey= im.convert(mode="L")
    #threshold= 160
    #image=grey.point(lambda x: 255 if x > threshold else 0)
    
    #configuration for pytesseract to operate with text extraction
    config = ('-l eng --oem 1 --psm 3')
    text = pytesseract.image_to_string(grey, config=config)
    
    return text

In [153]:
#extract address --in progress.... 
def extract_address(text):
    import re
    text=text
    regexp = "[0-9]{1,3} .+, .+, [A-Z]{2} [0-9]{5}"
    address = re.findall(regexp, text)
    return address 


#extract order

#def order():
    
#extract total

#def extract_total():
    
#makesure to have the total mapped into numbers if not extracted as a number, try to map the letters into numbers.





In [154]:

image=(r"images/large-receipt-image-dataset-SRD/1012-receipt.jpg")
text=extract_text(image)
print (text)

#14

HP Pho Ga
8930 Mission Dr. #102
pp ooee 91770
Phone (626)268-9999

 
     

bate: Apr 01, 2019 Time; 05:

Server: Admin

Bill; 59980 Table : 14

1 Pho Ga (Small) 8.00

1 Pho Ga (Large) 9.00

1 Gor Cuon 5.25
“Subtotal 22.25
TAX 211
Total 24.36
8. Service Gharge- 40% 8.90

Total $33 26

Suggested Tip : 15% (3,65)

18% (4.38)

20% (4.87)
Open Time : Apr 01, 2019 04:39PM
Printed By : Admin

Thank you for Coming !



In [157]:
import os 

dir=[]
directory=r"images/large-receipt-image-dataset-SRD/"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        dir.append(f)

In [159]:
texts=[]
for i in dir :
    text=extract_text(i)
    texts.append(text)

In [168]:
with open(r'text.txt', 'w',encoding="utf-8") as fp:
    for item in texts:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

Done


In [170]:
import pandas as pd 
df=pd.DataFrame(texts)
df.to_csv('texts.csv')

In [ ]:
from commonregex import CommonRegex

In [ ]:
parsed_text=CommonRegex(text)

In [ ]:
parsed_text.times

In [ ]:
parsed_text.phones

In [ ]:
parsed_text.street_addresses

In [ ]:
import usaddress 
usaddress.parse(text)

In [ ]:
pip install usaddress


In [8]:
#open camera 
#add automated conturing to it
#then add automated capture

import cv2

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")


ret, frame = cap.read()
frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
ret, thresh = cv2.threshold(frame, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 
contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
for c in contours:
        rect = cv2.boundingRect(c)
        if rect[2] < 100 or rect[3] < 100: continue
        #print cv2.contourArea(c)
        x,y,w,h = rect
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame,'Moth Detected',(x+w+10,y+h),0,0.3,(0,255,0))
        cv2.imshow('Input', frame)
        c = cv2.waitKey(1)
        if c == 27:
            break

cap.release()
cv2.destroyAllWindows()

In [ ]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
dilated = cv2.dilate(blurred, rectKernel)
edged = cv2.Canny(dilated, 100, 200, apertureSize=3)
contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
image_with_contours = cv2.drawContours(image.copy(), contours, -1, (0,255,0), 3)
largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
image_with_largest_contours = cv2.drawContours(image.copy(), largest_contours, -1, (0,255,0), 3)

largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
image_with_largest_contours = cv2.drawContours(image.copy(), largest_contours, -1, (0,255,0), 3)

def get_receipt_contour(contours):    
    # loop over the contours
    for c in contours:
        approx = approximate_contour(c)
        # if our approximated contour has four points, we can assume it is receipt's rectangle
        if len(approx) == 4:
            return approx
        
get_receipt_contour(largest_contours)

# use this !!!!

In [2]:
#capture largest contour 
#close 



import cv2

cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 9))
    dilated = cv2.dilate(blurred, rectKernel)
    edged = cv2.Canny(dilated, 100, 200, apertureSize=3)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    
    image_with_largest_contours = cv2.drawContours(edged.copy(), largest_contours, -1, (0,255,0), 3)

    largest_contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    image_with_largest_contours = cv2.drawContours(edged.copy(), largest_contours, -1, (0,255,0), 3)

    
    #scanned = wrap_perspective(image_with_receipt_contour.copy(), contour_to_rect(receipt_contour))
    
    
    for c in largest_contours:
        rect = cv2.boundingRect(c)
        if rect[2] < 100 or rect[3] < 100: continue
        #print cv2.contourArea(c)
        x,y,w,h = rect
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame,'Moth Detected',(x+w+10,y+h),0,0.3,(0,255,0))
        cv2.imshow('Input', frame)
    c = cv2.waitKey(1)
    if c == 27:
        break
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [203]:
def contour_to_rect(contour):
    pts = contour.reshape(4, 2)
    rect = np.zeros((4, 2), dtype = "float32")
    # top-left point has the smallest sum
    # bottom-right has the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # compute the difference between the points:
    # the top-right will have the minumum difference 
    # the bottom-left will have the maximum difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect / resize_ratio

In [204]:
def wrap_perspective(img, rect):
    # unpack rectangle points: top left, top right, bottom right, bottom left
    (tl, tr, br, bl) = rect
    # compute the width of the new image
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    # compute the height of the new image
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    # take the maximum of the width and height values to reach
    # our final dimensions
    maxWidth = max(int(widthA), int(widthB))
    maxHeight = max(int(heightA), int(heightB))
    # destination points which will be used to map the screen to a "scanned" view
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(rect, dst)
    # warp the perspective to grab the screen
    return cv2.warpPerspective(img, M, (maxWidth, maxHeight))

In [ ]:
scanned = wrap_perspective(original.copy(), contour_to_rect(receipt_contour))